<h1>Data Cookbook: Fundamentals of Web scraping & Data wrangling</h1>
<p>
Web scraping is an important part of data collection. A lot of the time, useful data won't be readily available to download. With web scraping, we can automate the process of gathering data from multiple web pages, saving us time and effort.

By scraping websites, we can access a wide range of data, including text, images, tables, and more. This data can be used for various purposes, such as market research, sentiment analysis, price comparison, content aggregation, and data analysis.
</p>

A basic website is set up for demo.<br>
run `python app.py` to start server before proceeding.

<h2>Basic web requests</h2>

<p>The libraries **requests**, **BeautifulSoup** and **lxml** are commonly used in Python for web scraping and web content retrieval tasks. 

requests: It is a powerful library that allows you to send HTTP requests to web pages and web services. With requests, you can easily retrieve HTML content, make GET and POST requests, handle cookies and sessions, and interact with web APIs. It's an essential tool for fetching web data and interacting with web resources programmatically.

BeautifulSoup: This library is a popular choice for parsing and navigating HTML and XML documents. It provides a convenient way to extract specific data from web pages by traversing the HTML document's structure. You can search for tags, access tag attributes, and extract text or data of interest. When used in combination with requests, BeautifulSoup becomes a powerful tool for web scraping and data extraction.

lxml is a sub library of pandas (so when importing it, just importing pandas is enough, assuming you have it installed) that is used to parse HTML efficiently.

Together, these libraries enable you to access web content, retrieve information, and perform data extraction tasks efficiently.</p>

In [29]:
# These are the dependencies, so if you don't have them, you'll need to install them with pip or whatever package manager you use
# pip install requests, bs4, pandas, lxml
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

This is the most basic usage of request. requests.get() send a GET request to the designated url, and returns the entire site's content as response. 

In [30]:
url = "http://127.0.0.1:5000/" # URL of the website
response = requests.get(url) # get the response from the website as a response object
print(response) # print the response object, though it's not what you'd expect

<Response [200]>


As you can see, a response object isn't useful to us just yet. It contains more information than just the site's source code. Depending on the site content, it could be text, json, image, api request, etc. Here we will just read the source code as text to see what it contains.

In [31]:
response_text = response.text # turn the response object into a string
print(response.text)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <h3>click link to see tables</h3>
    <a href="/table1">table 1</a>
    <a href="/table2">table 2</a>
    <a href="/">home</a>
    <h3>API guide</h3>
    <p>access subdomain 'table-data' and 'table2-data' to retrieve data directly</p>
    <p>stock image</p>
    <img alt="stock" src="/static/stock.png"></img>
</body>
</html>


Pulling image is a bit different, since you can't print it out directly. In order to keep the image, you must save it to a file.
We see an image present on the site: `<img alt="stock" src="/static/stock.png"></img>`. 

In [32]:
# Send a GET request to the image URL
response = requests.get(url + '/static/stock.png')

# Check if the request was successful
if response.status_code == 200:
    # Save the image locally
    with open("saved_image.png", "wb") as file:
        file.write(response.content)
        print("Image saved successfully.")
else:
    print("Failed to retrieve the image.")


Image saved successfully.


<h2>Parsing html for data</h2>
In order to parse the information we extracted from the site, and extract the table we want, we use the functions of BeautifulSoup.

In [33]:
table1 = requests.get(url + 'table1') # get the table as a repsonse object
soup = bs(table1.text, 'html.parser') # convert response object into text, then into a soup object throught html.parser
table1_processed = soup.find('table') # find any tables in the soup object
print(table1_processed)


<table>
<tr>
<th>Name</th>
<th>Age</th>
<th>City</th>
<th>Occupation</th>
<th>Salary</th>
<th>Hire Date</th>
</tr>
<tr>
<td>John Doe</td>
<td>25</td>
<td>New York</td>
<td>Software Engineer</td>
<td>$100,000</td>
<td>2022-09-01</td>
</tr>
<tr>
<td>Jane Smith</td>
<td>30</td>
<td>Los Angeles</td>
<td>Graphic Designer</td>
<td>$80,000</td>
<td>2022-09-02</td>
</tr>
<tr>
<td>Mike Johnson</td>
<td>35</td>
<td>Chicago</td>
<td>Marketing Manager</td>
<td>$120,000</td>
<td>2022-09-03</td>
</tr>
<tr>
<td>Sarah Williams</td>
<td>28</td>
<td>San Francisco</td>
<td>Data Analyst</td>
<td>$90,000</td>
<td>2022-09-04</td>
</tr>
<tr>
<td>David Brown</td>
<td>32</td>
<td>Miami</td>
<td>Product Manager</td>
<td>$110,000</td>
<td>2022-09-05</td>
</tr>
<tr>
<td>Emily Davis</td>
<td>27</td>
<td>Boston</td>
<td>UX Designer</td>
<td>$95,000</td>
<td>2022-09-06</td>
</tr>
<tr>
<td>Michael Johnson</td>
<td>33</td>
<td>Chicago</td>
<td>Marketing Manager</td>
<td>$120,000</td>
<td>2022-09-03</td>
</tr>
<tr>
<td

After parsing it and keeping only the table, we use the functions of lxml to parse it again and turn it into a pandas dataframe.

In [34]:
df_table1 = pd.read_html(str(table1_processed))[0]
print(df_table1.head())

             Name  Age           City         Occupation    Salary   Hire Date
0        John Doe   25       New York  Software Engineer  $100,000  2022-09-01
1      Jane Smith   30    Los Angeles   Graphic Designer   $80,000  2022-09-02
2    Mike Johnson   35        Chicago  Marketing Manager  $120,000  2022-09-03
3  Sarah Williams   28  San Francisco       Data Analyst   $90,000  2022-09-04
4     David Brown   32          Miami    Product Manager  $110,000  2022-09-05


C:\Users\steve\AppData\Local\Temp\ipykernel_22288\158322927.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_table1 = pd.read_html(str(table1_processed))[0]


<h2>Data wrangling</h2>
If you look at the site, you'll see there are two tables. There are no benefits for the two tables to be seperate, but for some reason they are. So in order to make using the data easier, we need to combine the tables into one.

In [35]:
# Getting the second table
table2 = requests.get(url + 'table1')
soup2 = bs(table1.text, 'html.parser')
table2_processed = soup2.find('table')
df_table2 = pd.read_html(str(table1_processed))[0]
print(df_table1.head())

             Name  Age           City         Occupation    Salary   Hire Date
0        John Doe   25       New York  Software Engineer  $100,000  2022-09-01
1      Jane Smith   30    Los Angeles   Graphic Designer   $80,000  2022-09-02
2    Mike Johnson   35        Chicago  Marketing Manager  $120,000  2022-09-03
3  Sarah Williams   28  San Francisco       Data Analyst   $90,000  2022-09-04
4     David Brown   32          Miami    Product Manager  $110,000  2022-09-05


C:\Users\steve\AppData\Local\Temp\ipykernel_22288\3038512959.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_table2 = pd.read_html(str(table1_processed))[0]


In [36]:
# Merging the two tables using built-in pandas function
combined_df = df_table1.merge(df_table2, on='Name')
print(combined_df)


               Name  Age_x         City_x       Occupation_x  Salary_x  \
0          John Doe     25       New York  Software Engineer  $100,000   
1        Jane Smith     30    Los Angeles   Graphic Designer   $80,000   
2      Mike Johnson     35        Chicago  Marketing Manager  $120,000   
3    Sarah Williams     28  San Francisco       Data Analyst   $90,000   
4       David Brown     32          Miami    Product Manager  $110,000   
5       David Brown     32          Miami    Product Manager  $110,000   
6       David Brown     32          Miami    Product Manager  $110,000   
7       David Brown     32          Miami    Product Manager  $110,000   
8       Emily Davis     27         Boston        UX Designer   $95,000   
9   Michael Johnson     33        Chicago  Marketing Manager  $120,000   
10    Anna Thompson     29  San Francisco       Data Analyst   $90,000   

   Hire Date_x  Age_y         City_y       Occupation_y  Salary_y Hire Date_y  
0   2022-09-01     25       New

Note that the 'Name' column is the only column that both tables have in common, so we use that as the key
If you want to merge on multiple columns, you can do something like this:
combined_df = df_table1.merge(df_table2, on=['Name', 'Age'])

<h2>Using APIs</h2>
While some sites doesn't provide ways to retrieve data, requiring you to scrape data off the site directly, some do offer built in methods of fetching data directly(and various other automation functions) in the form of API. APIs are used in a wide range of applications, from web and mobile app development to cloud services integration, IoT (Internet of Things), and more. They enable developers to leverage existing services and functionality, reducing development time and effort while promoting interoperability and flexibility in software ecosystems.

The provide sample site has a simplistic API built in, which allows users to retrieve the data tables in the form of JSON. If you nagivate directly to the links, you'll see a raw JSON table. This is much easier to process compared to extracting the tables from html.

In [37]:
# URLs for the APIs
url_table_data = url+'table-data'
url_table2_data = url+'table2-data'

# Making GET requests to the APIs
response_table_data = requests.get(url_table_data)
response_table2_data = requests.get(url_table2_data)

# Assuming the APIs return JSON data, parse the JSON into Python dictionaries
table_data = response_table_data.json()
table2_data = response_table2_data.json()
# Turn the JSON output into a Pandas DataFrame for further analysis
table_dp = pd.DataFrame(table_data)
table2_dp = pd.DataFrame(table2_data)
table_dp.head()
table2_dp.head()


,Name,Email,Phone Number,Department,Manager
0,John Doe,john.doe@example.com,123-456-7890,Engineering,Jane Wilson
1,Jane Smith,jane.smith@example.com,234-567-8901,Design,Robert Black
2,Mike Johnson,mike.johnson@example.com,345-678-9012,Marketing,Emily Green
3,Sarah Williams,sarah.williams@example.com,456-789-0123,Data Science,Michael Brown
4,David Brown,david.brown@example.com,567-890-1234,Product,Lisa White
